In [1]:
!pip install pytorch-lightning==1.6.5 spacy==2.2.4
!pip install pandas
!pip install -U sentence-transformers
!pip install transformers==4.17.0

You should consider upgrading via the '/Users/vitalii.mishchenko/Documents/experiments/2302-nlp-course/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/vitalii.mishchenko/Documents/experiments/2302-nlp-course/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/vitalii.mishchenko/Documents/experiments/2302-nlp-course/venv/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/Users/vitalii.mishchenko/Documents/experiments/2302-nlp-course/venv/bin/python -m pip install --upgrade pip' command.


In [2]:
from torch import nn
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset, random_split
from collections import Counter
import en_core_web_md
import numpy as np
import pytorch_lightning as pl
import spacy
import torch
import torch.nn.functional as F
import torchmetrics
import pandas as pd
from transformers import BertTokenizer, BertModel, BertTokenizerFast, DistilBertTokenizer, DistilBertModel
from sentence_transformers import SentenceTransformer

In [3]:
# Fix the random seed so that we get consistent results
torch.manual_seed(0)
np.random.seed(0)

In [4]:
import tarfile
import os
import csv

DIRECTORY_NAME="classification"
TRAIN_FILE="classification/empatheticdialogues/train.csv"
VALIDATION_FILE="classification/empatheticdialogues/valid.csv"
TEST_FILE="classification/empatheticdialogues/test.csv"


def download_dataset():
  """
  Download the dialog dataset. The tarball contains three files: train.csv, valid.csv, test.csv
  """

  # if running locally, install wget before by running: "brew install wget"
  # "!pip install wget" won't help as it installs python module not cli
  !wget 'https://dl.fbaipublicfiles.com/parlai/empatheticdialogues/empatheticdialogues.tar.gz'
  # !wget help
  if not os.path.isdir(DIRECTORY_NAME):
    !mkdir classification
  tar = tarfile.open('empatheticdialogues.tar.gz')
  tar.extractall(DIRECTORY_NAME)
  tar.close()

# Expensive operation so we should just do this once
download_dataset()

--2023-03-03 15:18:30--  https://dl.fbaipublicfiles.com/parlai/empatheticdialogues/empatheticdialogues.tar.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 104.22.74.142, 104.22.75.142, 172.67.9.4, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|104.22.74.142|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 28022709 (27M) [application/gzip]
Saving to: ‘empatheticdialogues.tar.gz’

empatheticdialogues 100%[===================>]  26.72M  9.56MB/s    in 2.8s    

2023-03-03 15:18:34 (9.56 MB/s) - ‘empatheticdialogues.tar.gz’ saved [28022709/28022709]



In [5]:
import glob
glob.glob(f"{DIRECTORY_NAME}/**/*.csv", recursive=True)

['classification/empatheticdialogues/valid.csv',
 'classification/empatheticdialogues/test.csv',
 'classification/empatheticdialogues/train.csv']

In [6]:
# See the parse_dataset function below for short explanation.
df = pd.read_csv(TRAIN_FILE, sep='\n', header=None)
df = df[0].str.split(',', expand=True)
new_header = df.iloc[0]
df = df[1:]
df.columns = new_header
df.head(5)

,conv_id,utterance_idx,context,prompt,speaker_idx,utterance,selfeval,tags
1,hit:0_conv:1,1,sentimental,I remember going to the fireworks with my best...,1,I remember going to see the fireworks with my ...,5|5|5_2|2|5,
2,hit:0_conv:1,2,sentimental,I remember going to the fireworks with my best...,0,Was this a friend you were in love with_comma_...,5|5|5_2|2|5,
3,hit:0_conv:1,3,sentimental,I remember going to the fireworks with my best...,1,This was a best friend. I miss her.,5|5|5_2|2|5,
4,hit:0_conv:1,4,sentimental,I remember going to the fireworks with my best...,0,Where has she gone?,5|5|5_2|2|5,
5,hit:0_conv:1,5,sentimental,I remember going to the fireworks with my best...,1,We no longer talk.,5|5|5_2|2|5,


In [7]:
# "sentimental" to "<id>"
label_to_integer = dict()
# "<id>" to "sentimental"
integer_to_label = dict()

for ix, label in enumerate(df["context"].unique()):
  label_to_integer[label] = ix
  integer_to_label[ix] = label

In [8]:
def parse_dataset(file_path, sample=5000):
  # Our dataset file is a csv with varying input lengths, therefore we load the
  # file at once, we have to split it up into separate steps:
  # 1. Read each row as a single column row
  df = pd.read_csv(file_path, sep = '\n', header = None)
  # 2. Split up each row into separate columns
  df = df[0].str.split(',', expand = True)
  # 3. Set the header by using the first row
  new_header = df.iloc[0]
  df = df[1:]
  df.columns = new_header

  # Machine learning cannot work with categorical labels like "surprised" or
  # "excited". Therefore, we convert these tokens into a number.
  df["target"] = df["context"].apply(lambda x: label_to_integer[x])
  df["feature"] = df["prompt"] + " " + df["utterance"]

  # We only need the column "**feature**" created from column
  # "**prompt**" + "**utterance**" and the column "**target**".
  return df[["target", "feature"]].sample(n = sample, random_state = 0).values

In [9]:
training_data = parse_dataset(TRAIN_FILE, sample = 40000)
validation_data = parse_dataset(VALIDATION_FILE, sample = 4000)
test_data = parse_dataset(TEST_FILE, sample = 4000)

print('Shape of training dataset: ({rows}, {cols})'.format(rows=len(training_data), cols=len(training_data[0])))
print('Shape of validation dataset: ({rows}, {cols})'.format(rows=len(validation_data), cols=len(validation_data[0])))
print('Shape of test dataset: ({rows}, {cols})'.format(rows=len(test_data), cols=len(test_data[0])))

Shape of training dataset: (40000, 2)
Shape of validation dataset: (4000, 2)
Shape of test dataset: (4000, 2)


In [10]:
class ClassificationDataset(Dataset):
  """Creates an pytorch dataset to consume our pre-loaded csv data

  Reference: https://pytorch.org/tutorials/beginner/basics/data_tutorial.html
  """
  def __init__(self, data, vectorizer):
    self.dataset = data
    # Vectorizer needs to implement a vectorize function that returns vector and tokens
    # 🌟🌟🌟 Pay extra attention here since you'll have to work on this in the models 🌟🌟🌟
    self.vectorizer = vectorizer

  def __len__(self):
    return len(self.dataset)

  def __getitem__(self, idx):
    (label, sentence) = self.dataset[idx]
    sentence_vector, sentence_tokens = self.vectorizer.vectorize(sentence)
    return {
      "vectors": sentence_vector,
      "label": label,
      "tokens": sentence_tokens, # for debugging only
      "sentence": sentence # for debugging only
    }

In [11]:
class ClassificationDataModule(pl.LightningDataModule):
  """LightningDataModule: Wrapper class for the dataset to be used in training
  """
  def __init__(self, vectorizer, params):
    super().__init__()
    self.params = params
    self.classification_train = ClassificationDataset(training_data, vectorizer)
    self.classification_val = ClassificationDataset(validation_data, vectorizer)
    self.classification_test = ClassificationDataset(test_data, vectorizer)

  # Function to convert the input raw data from the dataset into model input.
  # 🌟🌟🌟 Pay extra attention here since you'll have to work on this in the models 🌟🌟🌟
  def collate_fn(self, batch):
    # Embedding layers need the inputs to be integer, so we need to add this special case here.
    if self.params.integer_input:
      word_vector = [torch.LongTensor(item["vectors"]) for item in batch]
      sentence_vector = pad_sequence(word_vector, batch_first=True, padding_value=0)
    else:
      sentence_vector = torch.stack([torch.Tensor(item["vectors"]) for item in batch])
    labels = torch.LongTensor([item["label"] for item in batch])
    return {"vectors": sentence_vector, "labels": labels, "sentences": [item["sentence"] for item in batch]}

  # Training dataloader .. will reset itself each epoch
  def train_dataloader(self):
    return DataLoader(self.classification_train, batch_size=self.params.batch_size, collate_fn=self.collate_fn)

  # Validation dataloader .. will reset itself each epoch
  def val_dataloader(self):
    return DataLoader(self.classification_val, batch_size=self.params.batch_size, collate_fn=self.collate_fn)

  # Test dataloader .. will reset itself each epoch
  def test_dataloader(self):
    return DataLoader(self.classification_test, batch_size=self.params.batch_size, collate_fn=self.collate_fn)

In [12]:
class EmotionClassifier(pl.LightningModule):
  def __init__(self, model, params):
    super().__init__()
    self.model = model
    self.params = params
    self.accuracy = torchmetrics.Accuracy(task='multiclass', num_classes=params.num_classes)

  # will be called automatically by "training_step", "validation_step", etc.
  def forward(self, x):
    return self.model(x)

  def training_step(self, batch, batch_idx):
    x = batch["vectors"]
    y = batch["labels"]
    y_hat = self(x)
    loss = F.cross_entropy(y_hat, y, reduction='mean')
    self.log_dict(
      {'train_loss': loss},
      batch_size=self.params.batch_size,
      prog_bar=True
    )
    return loss

  def validation_step(self, batch, batch_nb):
    x = batch["vectors"]
    y = batch["labels"]
    y_hat = self(x)
    val_loss = F.cross_entropy(y_hat, y, reduction='mean')
    predictions = torch.argmax(y_hat, dim=1)
    self.log_dict(
      {
        'val_loss': val_loss,
        'val_accuracy': self.accuracy(predictions, y)
      },
      batch_size=self.params.batch_size,
      prog_bar=True
    )
    return val_loss

  def test_step(self, batch, batch_nb):
    x = batch["vectors"]
    y = batch["labels"]
    y_hat = self(x)
    test_loss = F.cross_entropy(y_hat, y, reduction='mean')
    predictions = torch.argmax(y_hat, dim=1)
    self.log_dict(
      {
        'test_loss': test_loss,
        'test_accuracy': self.accuracy(predictions, y)
      },
      batch_size=self.params.batch_size,
      prog_bar=True
    )
    return test_loss

  def predict_step(self, batch, batch_idx):
    y_hat = self.model(batch["vectors"])
    predictions = torch.argmax(y_hat, dim=1)
    return {'logits':y_hat, 'predictions': predictions, 'labels': batch["labels"], 'sentences': batch['sentences']}

  def configure_optimizers(self):
    optimizer = torch.optim.Adam(self.parameters(), lr=self.params.learning_rate)
    return optimizer

In [13]:
def trainer(model, params, vectorizer):
  # Create a pytorch trainer
  trainer = pl.Trainer(max_epochs=params.max_epochs, check_val_every_n_epoch=1)

  # Initialize our data loader with the passed vectorizer
  data_module = ClassificationDataModule(vectorizer, params)

  # Instantiate a new model
  emotionClassifier = EmotionClassifier(model, params)

  # Train and validate the model
  trainer.fit(emotionClassifier, data_module.train_dataloader(), val_dataloaders=data_module.val_dataloader())

  # Test the model
  trainer.test(emotionClassifier, data_module.test_dataloader())

  # Predict on the same test set to show some output
  output = trainer.predict(emotionClassifier, data_module.test_dataloader())

  for i in range(2):
    print("-----------")
    print("Sentence: ", output[1]['sentences'][i])
    print("Predicted Emotion: ", integer_to_label[output[1]['predictions'][i].item()])
    print("Actual Label: ", integer_to_label[output[1]['labels'][i].item()])

In [16]:
class WordVectorClassificationModel(torch.nn.Module):
  def __init__(self, word_vec_dimension, num_classes):
    super().__init__()
    self.classes = num_classes
    self.linear_layer = torch.nn.Linear(word_vec_dimension, num_classes)

  # 🌟🌟🌟 Pay extra attention here since you'll have to work on this in the models 🌟🌟🌟
  def forward(self, batch):
    """Projection from word_vec_dim to n_classes

    Batch is of shape (batch_size, max_seq_len, word_vector_dim)
    """
    return self.linear_layer(batch)

In [1]:
class BertVectorizer:
  def __init__(self):
    # https://huggingface.co/prajjwal1/bert-tiny
    # small bert model for better performance
    model_name = 'prajjwal1/bert-tiny'
    self.tokenizer = DistilBertTokenizer.from_pretrained(model_name)
    self.model = DistilBertModel.from_pretrained(model_name)

  def vectorize(self, sentence):
    """
    Given a sentence, tokenize it and reference pre-trained word vector for each token.

    Returns a tuple of sentence_vector and list of text tokens
    """

    inputs = self.tokenizer(sentence, return_tensors="pt")
    outputs = self.model(**inputs)
    model_output = outputs['last_hidden_state'].detach()
    sentence_vector = np.mean(np.array(model_output[-1]), axis=0)

    token_ids = inputs.input_ids[-1].tolist()
    sentence_tokens = [self.tokenizer.decode(token_id) for token_id in token_ids]

    return sentence_vector, sentence_tokens

class HParams:
  batch_size: int = 32
  integer_input: bool = False
  word_vec_dimension: int = 128
  num_classes: int = 32
  learning_rate: float = 0.001
  max_epochs: int = 4

bertVectorizer = BertVectorizer()
model = WordVectorClassificationModel(
  HParams.word_vec_dimension,
  HParams.num_classes
)

# test_accuracy         0.1745000034570694
# test_loss             3.06999135017395
trainer(
  model=model,
  params=HParams,
  vectorizer=bertVectorizer)

NameError: name 'DistilBertTokenizer' is not defined

In [19]:
class BertSentenceVectorizer:
  def __init__(self):
    self.model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

  def vectorize(self, sentence):
    """
    Given a sentence, tokenize it and reference pre-trained word vector for each token.

    Returns a tuple of sentence_vector and list of text tokens
    """

    sentence_vectors = self.model.encode(sentence)
    sentence_tokens = []

    return sentence_vectors, sentence_tokens

class HParams:
  batch_size: int = 32
  integer_input: bool = False
  word_vec_dimension: int = 384
  num_classes: int = 32
  learning_rate: float = 0.001
  max_epochs: int = 4

bertSentenceVectorizer = BertSentenceVectorizer()
model = WordVectorClassificationModel(
  HParams.word_vec_dimension,
  HParams.num_classes
)

# test_accuracy         0.4097500145435333
# test_loss             1.9137377738952637
trainer(
  model=model,
  params=HParams,
  vectorizer=bertSentenceVectorizer)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name     | Type                          | Params
-----------------------------------------------------------
0 | model    | WordVectorClassificationModel | 12.3 K
1 | accuracy | MulticlassAccuracy            | 0     
-----------------------------------------------------------
12.3 K    Trainable params
0         Non-trainable params
12.3 K    Total params
0.049     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/Users/vitalii.mishchenko/Documents/experiments/2302-nlp-course/venv/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,
/Users/vitalii.mishchenko/Documents/experiments/2302-nlp-course/venv/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.4097500145435333
        test_loss           1.9137377738952637
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/Users/vitalii.mishchenko/Documents/experiments/2302-nlp-course/venv/lib/python3.7/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:245: PossibleUserWarning: The dataloader, predict_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 10 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  category=PossibleUserWarning,


Predicting: 1250it [00:00, ?it/s]

-----------
Sentence:  When I got falsely accused of eating my roommate's ice cream last night_comma_ I was completely outraged! I don't even eat ice cream because I'm lactose intolerant_comma_ and she knows that! I wonder who actually ate it then.
Predicted Emotion:  angry
Actual Label:  furious
-----------
Sentence:  I found out that my childhood cat passed away yesterday  Thats sad. I love cats
Predicted Emotion:  sad
Actual Label:  sad
